<a href="https://colab.research.google.com/github/shahchhatru/AI_colab_notebooks/blob/main/transformerfromscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing required libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import torchtext
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)


1.13.0+cu116


# Create word embeddings
First of all we need to convert each word to an embedding vector.Embedding vectors create more semantic representation of word.
Suppose each embedding vector is of size 512 and our embedding matrix is size of 100*512 .The matrix will be learned on training and during inference each word will be mapped to the corresponding 512d vector. Suppose we have a batch size of 32 and sequence of length 10 the the output will be 32x10x512 matrix.

In [ ]:
# embedding class
class Embedding(nn.Module):
  def __init__(self,vocab_size,embed_dim):
    """
    arg vocab size :size of vocabulary
    embeded_dim:dimension of embeddings.
    """
    super(Embedding,self).__init__()
    self.embed=nn.Embedding(vocab_size,embed_dim)

  def forward(self,x):
    """
    args
    x :input_vector

    returns
    out:dimension_of_dim

    """
    out=self.embed(x)
    return out



# positional encoding
Next step is to generate positional encoding . Inorder for the model to make sense of the word it needs to know about the meaning of the sentence and it's position in the sentence.

In attention is all you need paper . The author used the following functions to create positional encodings.
on odd time steps a cosine function is used and on even time steps a
sine function is used.
```
\PE_(pos,2i)=sin(pos/10000^(2*i/d))\

\PE_(pos,2i+1)=cos(pos/10000^(2*i/d))\

```
pos -> refers to order in the sentence
i -> refers to position along embedding vector dimension

positional embedding will generate a matrix similar to embedding matrix.
it will create a matrix of dimension sequence_length x embedding_dimension
for each word we will get a embedding vector which is of dimension 1x512 and similarly we will also find a corresponding positional vector of 1x512 to get a 1x512 vector for each word.

or eg: if we have batch size of 32 and seq length of 10 and let embedding dimension be 512. Then we will have embedding vector of dimension 32 x 10 x 512. Similarly we will have positional encoding vector of dimension 32 x 10 x 512. Then we add both.


In [ ]:
## register buffer in Pytorch ->
# If you have parameters in your model, which should be saved and restored in the state_dict,
# but not trained by the optimizer, you should register them as buffers.



In [ ]:
class PositionalEmbedding(nn.Module):
    def __init__(self,max_seq_len,embed_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super(PositionalEmbedding, self).__init__()
        self.embed_dim = embed_model_dim

        pe = torch.zeros(max_seq_len,self.embed_dim)
        for pos in range(max_seq_len):
            for i in range(0,self.embed_dim,2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/self.embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/self.embed_dim)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """

        # make embeddings relatively larger
        x = x * math.sqrt(self.embed_dim)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)
        return x

#self attention and mutihead attention
### self attention
suppose we have the sentence dog is crossing the street 'Dog is crossing the street because it saw the kitten' here we know it refers to Dog but the machine won't understand it.
As model proceeses each word, self attention allows it to look at other positions in the input sequence for clues. It will creates a vector based on dependency of each word with the other.

Let us go through a step by step illustration of self attention.

Step 1: The first step in calculating self-attention is to create three vectors from each of the encoder’s input vectors (in this case, the embedding of each word). So for each word, we create a Query vector, a Key vector, and a Value vector. Each of the vector will be of dimension 1x64.
Since we have a multihead attention we will have 8 self attention heads.I will explain the code with 8 attention head in mind.